Upload files to colab

Translation function

In [0]:
#!/usr/bin/env python
# coding: utf-8

# Translate sentences to English and save mapping dicitionary as json.

# Based on https://github.com/ruanchaves/assin/blob/master/sources/translate.py

# Parameters:
# *   sentences =
# *   dictpath =  json dictionary file mapping original sentence with translation
# 
# 

# In[ ]:


from google.cloud import translate_v2 as translate
from os.path import isfile
from os import environ
import six
import json

def translate2dict(sentences, dictpath):
    environ['GOOGLE_APPLICATION_CREDENTIALS'] = './key.json'
    translate_client = translate.Client()

    def translation(text):
        if isinstance(text, six.binary_type):
            text = text.decode('utf-8')
        result = translate_client.translate(text,'pt-br')

        return result['translatedText']

    if not isfile(dictpath):
        with open(dictpath, 'w') as f:
            json.dump({}, f)

    for item in sentences:
        with open(dictpath) as f:
            translations = json.load(f)
        
        try:
            translations[item]
        except KeyError:
            translations[item] = translation(item)
            
            with open(dictpath, 'w+') as f:
                json.dump(translations, f)

Prepare input

In [16]:
from pprint import pprint
import pandas as pd
import os

sentences = list()
label_idx = [
             '#1 String',
             '#2 String',
            ]
length = 0

for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    print(f) 
    #print(table.head())
    #print('\n'*3)
    
    for idx in label_idx:
        label = table[idx].copy()
        #print(label)
        #print('\n'*5)

        sentences.extend(list(label))
        length = length + label.size


assert length == len(sentences)
sentences = set(sentences)

train.tsv
test.tsv
dev.tsv


Translate

In [0]:
translate2dict(sentences,  'mrpc.json')

Replace with translation

In [0]:
from os import mkdir
import json

with open('mrpc.json') as reader:
    dictionary = json.load(reader)

mkdir('translation')
for f in ['train.tsv', 'test.tsv', 'dev.tsv']:
    table = pd.read_csv(f, sep = '\t', quoting=3)
    for idx in label_idx:
        table[idx] = table[idx].map(dictionary)
    table.to_csv(f'translation/{f}', index = None, header = None, sep = '\t')